In [1]:
import tensorflow as tf
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,"
from os.path import isdir
from os import mkdir
from shutil import copyfile
import time
from PIL import Image
import pickle
from spoofing_ld_conv.SpoofLineConvTf import SpoofLineConvTf
from mklib.nn.tfnet.tfresnet import TfResNet
from tensorflow.keras.regularizers import l1, l2
from tensorflow.core.protobuf import config_pb2
#import sys
#sys.path.append('/home/macul/libraries/mk_utils/mklib/nn/tfnet/')
#sys.path.append('/home/macul/libraries/mk_utils/mklib/nn/tfloss/')
#sys.path.append('/home/macul/libraries/mk_utils/tf_spoof/')
#sys.path.append('/home/macul/libraries/mk_utils/spoofing_ld_conv/')
from tf_spoof.config import net_config_ld_conv as config



In [2]:
SpoofVal = SpoofLineConvTf(config)

In [ ]:
weights = tf.get_variable(name='embedding_weights', shape=(2,3),
                                  initializer=tf.contrib.layers.xavier_initializer(uniform=False), dtype=tf.float32, trainable=True)
embedding_norm = tf.math.l2_normalize([[0., 0., 0., 0., 0., 0., 0., 0., 0.]], axis=1)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    a1=sess.run(weights)
    a2=sess.run(embedding_norm)
    print(a2,a1)

In [ ]:
img_ph1 = tf.placeholder(name='data1',shape=[1,112,112,3], dtype=tf.float32)
label_ph1 = tf.placeholder(name='label1',shape=[1,], dtype=tf.int64)
config1 = {'Regularizer': config.Regularizer, 'Activation': config.Activation,
           'Arc_margin_scale': 64.0, 'Arc_margin_angle': 0.5,
           'loss_type': 'arc'}

embedding, logit, inference_loss, pred, acc  = TfResNet.build_with_loss(img_ph1, label_ph1, 3, 128, [3], [64,32], training=True, loss_config=config1, initializer=config.Initializer, scope='test2')

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    embedding1, logit1, inference_loss1, pred1, acc1 = sess.run([embedding, logit, inference_loss, pred, acc], feed_dict={img_ph1: np.random.rand(1,112,112,3), label_ph1:[2]})

In [ ]:
print(embedding1, logit1, inference_loss1, pred1, acc1)

In [ ]:
spfLC = SpoofLineConvTf(config)

In [ ]:
#spfLC.test_dataset()

In [ ]:
iterator, next_element = spfLC.build_dataset(config.TRAIN_REC, 1, config.NUM_CLASSES, 
                                             target_size=config.Image_Height, fill_value=config.Fill_Value, 
                                             ratio_h=config.Fill_Ratio_H, ratio_w=config.Fill_Ratio_W, 
                                             training=True)
logit, inference_loss, pred, acc, embeddings, _, _= spfLC.build_net(training=True, reuse=None)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(iterator.initializer)
    images_train, labels_train = sess.run(next_element)
    images_train2, labels_train2 = sess.run(next_element)
    feed_dict = {spfLC.images1_ph: images_train, spfLC.labels1_ph: labels_train,
                 spfLC.images2_ph: images_train2, spfLC.labels2_ph: labels_train2}
    logit_t, inference_loss_t, pred_t, acc_t, embeddings_t = \
                        sess.run([logit, inference_loss, pred, acc, embeddings],
                                  feed_dict=feed_dict,
                                  options=config_pb2.RunOptions(report_tensor_allocations_upon_oom=True))

In [ ]:
print(logit_t)
print(inference_loss_t)
print(pred_t)
print(acc_t)

In [ ]:
embeddings1, logit1, inference_loss1, pred1, acc1 = spfLC.build_net(training=False, reuse=True)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(iterator.initializer)
    images_train, labels_train = sess.run(next_element)
    feed_dict = {spfLC.images_val_ph: images_train, spfLC.labels_val_ph: labels_train}
    logit1_t, inference_loss1_t, pred1_t, acc1_t, embeddings1_t = \
                        sess.run([embeddings1, logit1, inference_loss1, pred1, acc1],
                                  feed_dict=feed_dict,
                                  options=config_pb2.RunOptions(report_tensor_allocations_upon_oom=True))

In [ ]:
print(logit1_t)
print(inference_loss1_t)
print(pred1_t)
print(acc1_t)

In [ ]:
with tf.Session() as sess:
    sess.run(iterator.initializer)
    images_train, labels_train = sess.run(next_element)
    
    print(images_train, labels_train)

In [ ]:
print(images_train.shape)

In [ ]:
img = Image.fromarray(images_train[0,...].astype(np.uint8), 'RGB')
img.show()

In [ ]:
img_ph = tf.placeholder(name='data',shape=[2,224,224,3], dtype=tf.float32)
img_ph1 = tf.placeholder(name='data1',shape=[2,224,224,3], dtype=tf.float32)

act3_1= TfResNet.build_modules(img_ph, 512, [1,1,1,1,1], [8,8,8,16,16,8], training=True, initializer=Initializer, scope='test1')
act3_2= TfResNet.build_modules(img_ph, 512, [1,1,1,1,1], [8,8,8,16,16,8], training=True, initializer=Initializer, scope='test1', reuse=True)
act3=tf.concat([act3_1,act3_2],3)

act3_3=tf.layers.conv2d( inputs=act3, padding='same', kernel_size=(3, 3), strides=(2, 2), filters=32,use_bias=False, name='stem_conv1')
act3_4=tf.layers.conv2d( inputs=act3_3, padding='same', kernel_size=(3, 3), strides=(2, 2), filters=16,use_bias=False, name='stem_conv2')
act3_5=tf.layers.conv2d( inputs=act3_4, padding='valid', kernel_size=(2, 2), strides=(1, 1), filters=1,use_bias=False, name='stem_conv3')
act3_5=tf.squeeze(act3_5)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    act = sess.run(act3_5, feed_dict={img_ph: np.random.rand(2,224,224,3),img_ph1: np.random.rand(2,224,224,3)})
    
print(act)

In [ ]:
img_ph = tf.placeholder(name='data',shape=[1,112,112,3], dtype=tf.float32)

embedding0, act2_1= TfResNet.build_embedding(img_ph, 512, [3], [64,32], training=True, initializer=Initializer, scope='test1')

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    emb,act2 = sess.run([embedding0,act2_1], feed_dict={img_ph: np.random.rand(1,112,112,3)})

In [ ]:
print(emb[0],act2)

In [ ]:
label_ph2 = tf.placeholder(name='label2',shape=[3,], dtype=tf.int64)
label_ph3 = tf.placeholder(name='label3',shape=[3,], dtype=tf.int64)

label_x = tf.cast(tf.equal(label_ph2,label_ph3), tf.int64)

In [ ]:
with tf.Session() as sess:
    lb = sess.run(label_x, feed_dict={label_ph2:[1,1,1], label_ph3:[1,0,2]})
print(lb)

In [ ]:
img_ph1 = tf.placeholder(name='data1',shape=[2,112,112,3], dtype=tf.float32)
img = tf.image.resize_bilinear(img_ph1, (64, 64))
with tf.Session() as sess:
    emb= sess.run(img, feed_dict={img_ph1: np.random.rand(2,112,112,3)})
print(emb.shape)

In [ ]:
mask = np.ones([1,112,112,3])
mask[0,0:10,0:10,:] = 0
mask_tf=tf.convert_to_tensor(mask, dtype=tf.float32)

img_ph1 = tf.placeholder(name='data1',shape=[1,112,112,3], dtype=tf.float32)
img_mask = tf.multiply(mask_tf, img_ph1)
with tf.Session() as sess:
    emb= sess.run(img_mask, feed_dict={img_ph1: np.random.rand(1,112,112,3)})
print(mask_tf.dtype)

In [ ]:
def img_fill_center(img, fill_value=127, ratio_h=0.4, ratio_w=0.35):

    def fill_c(img, fill_value, ratio_h, ratio_w):
        im = img.copy()
        b,h,w,c = img.shape
        cp_h = int(h*ratio_h)
        cp_w = int(w*ratio_w)
        cp_x = int((w-cp_w)/2)
        cp_y = int((h-cp_h)/2)
        im[:,cp_y:cp_y+cp_h,cp_x:cp_x+cp_w,:] = fill_value

        return im

    return tf.cast(tf.py_func(fill_c, [img, fill_value, ratio_h, ratio_w], tf.float32),tf.float32)

a=np.random.rand(1,112,112,3)
img_ph1 = tf.placeholder(name='data1',shape=[1,112,112,3], dtype=tf.float32)
img_mask=img_fill_center(img_ph1)
with tf.Session() as sess:
    emb= sess.run(img_mask, feed_dict={img_ph1: a})
print(np.sum(emb==127))

In [ ]:
print(emb[0,33,:,0])

In [ ]:
a = np.ones([1,112,112,3])
a[0,0:10,0:10,:] = 0
at=tf.convert_to_tensor(a)
with tf.Session() as sess:
    emb= sess.run(at)
print(emb)

In [ ]:
a=100
b=tf.cast(a,tf.int64)
with tf.Session() as sess:
    bb = sess.run(b)
print(bb)

In [ ]:
tfSpoofLd = TfSpoofLd()

In [ ]:
sess, lines_in, pred_out, names, graph_nodes = tfSpoofLd.load_pb(pb_path)

In [ ]:
prediction = tfSpoofLd.eval(sess, lines_in, pred_out)
print(prediction)

In [ ]:
print(nodes)

In [ ]:
print(names)

In [ ]:
np.random.rand(1,2)

In [ ]:
a=np.array([[[[9.999995e-01, 5.332333e-07]]]])

In [ ]:
a.shape

In [ ]:
np.squeeze(a)

In [ ]:
a_ph = tf.placeholder(name='a',shape=[2,3], dtype=tf.int64)
b_ph = tf.placeholder(name='b',shape=[2,1], dtype=tf.int64)
c=tf.multiply(b_ph,a_ph)

In [ ]:
with tf.Session() as sess:
    rst = sess.run(c, feed_dict={a_ph: np.array([[1,1,1],[1,1,1]]), b_ph: np.array([[0],[1]])})
    print(rst)